# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd84098ffd0>
├── 'a' --> tensor([[-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245]])
└── 'x' --> <FastTreeValue 0x7fd84098f580>
    └── 'c' --> tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                        [-0.7221,  0.2501, -1.3356, -0.3699],
                        [-0.7103,  0.4527, -1.1712, -1.5749]])

In [4]:
t.a

tensor([[-0.1177,  0.0695, -1.0330],
        [-1.7521,  0.5552,  0.3245]])

In [5]:
%timeit t.a

63 ns ± 0.164 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd84098ffd0>
├── 'a' --> tensor([[-0.9805, -0.2984,  1.4127],
│                   [ 1.3845, -1.0474,  0.4162]])
└── 'x' --> <FastTreeValue 0x7fd84098f580>
    └── 'c' --> tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                        [-0.7221,  0.2501, -1.3356, -0.3699],
                        [-0.7103,  0.4527, -1.1712, -1.5749]])

In [7]:
%timeit t.a = new_value

68.2 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245]]),
    x: Batch(
           c: tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                      [-0.7221,  0.2501, -1.3356, -0.3699],
                      [-0.7103,  0.4527, -1.1712, -1.5749]]),
       ),
)

In [10]:
b.a

tensor([[-0.1177,  0.0695, -1.0330],
        [-1.7521,  0.5552,  0.3245]])

In [11]:
%timeit b.a

58.8 ns ± 0.0777 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.8575,  1.4628, -0.8571],
               [ 0.0098, -0.8673,  1.0717]]),
    x: Batch(
           c: tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                      [-0.7221,  0.2501, -1.3356, -0.3699],
                      [-0.7103,  0.4527, -1.1712, -1.5749]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

819 ns ± 0.188 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd788bd6ca0>
├── 'a' --> tensor([[[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]],
│           
│                   [[-0.1177,  0.0695, -1.0330],
│                    [-1.7521,  0.5552,  0.3245]]])
└── 'x' --> <FastTreeValue 0x7fd788bd6b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd8409229d0>
├── 'a' --> tensor([[-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245],
│                   [-0.1177,  0.0695, -1.0330],
│                   [-1.7521,  0.5552,  0.3245]])
└── 'x' --> <FastTreeValue 0x7fd840922ee0>
    └── 'c' --> tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                        [-0.7221,  0.2501, -1.3356, -0.3699],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 64.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]],
       
               [[-0.1177,  0.0695, -1.0330],
                [-1.7521,  0.5552,  0.3245]]]),
    x: Batch(
           c: tensor([[[ 0.7492, -0.4632,  0.0292,  1.0156],
                       [-0.7221,  0.2501, -1.3356, -0.3699],
                       [-0.7103,  0.4527, -1.1712, -1.5749]],
         

In [26]:
%timeit Batch.stack(batches)

79.5 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245],
               [-0.1177,  0.0695, -1.0330],
               [-1.7521,  0.5552,  0.3245]]),
    x: Batch(
           c: tensor([[ 0.7492, -0.4632,  0.0292,  1.0156],
                      [-0.7221,  0.2501, -1.3356, -0.3699],
                      [-0.7103,  0.4527, -1.1712, -1.5749],
                      [ 0.7492, -0.4632,  0.0292,  1.0156],
                      [-0.7221,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 777 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
